In [ ]:
#### To-Do: 
# Work on the Hover
# Put all visualizations within one script
# Implememt Box Charts
# Style the Quantity of Hot Days with split color background

In [220]:
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
import pandas as pd
import sqlite3

############ Configure the data
def main():
    output_notebook()
    conn = sqlite3.connect('weather.db')
    weather = pd.read_sql('select DATE, (TMAX+TMIN)/2 Temp, TMAX high, TMIN low, YEAR, MONTH, DAY from weather where station = "USW00093821"', conn)
    
    years = []
    for name, group in month_group:
        years.append(temp_plots(name, group))
    show(column(Div(text="<h2>Quantity of Hottest Days</h2>")))
    show(hottest_day_counter())
    show(column(Div(text="<h2>Months Temp by Year</h2>")))
    show(gridplot(data_grid(years)))

    # Retrieve temp range from sql results
    max_temp, min_temp = (weather.high.max() + 10), weather.low.min()

def hottest_day_counter():
    temp_uphi = weather.loc[weather.groupby(['MONTH', 'DAY'])['high'].idxmax()]
    temp_dlo = weather.loc[weather.groupby(['MONTH', 'DAY'])['low'].idxmax()]

    temp_uhi_data = temp_uphi.groupby('YEAR').size()
    temp_dlo_data = temp_dlo.groupby('YEAR').size() * -1

    # hover = HoverTool()
    # hover.tooltips = [
    #     ("date", "$index"),
    #     ("(x,y)", "($x, $y)"),
    #     ("radius", "@radius"),
    #     ("fill color", "$color[hex, swatch]:fill_color"),
    #     ("foo", "@foo"),
    #     ("bar", "@bar"),
    # ]
    ### This is the Num_of_Hottest_Days Graph
    p = figure(width=1000, plot_height=500)
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_color='red'
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Years'
    p.yaxis.axis_label = "Num of Lowest Lows \t|| \tNum of Highest Highs"
    p.line(temp_uhi_data.index, temp_uhi_data.values, line_width=2, color='red', legend="Max Highs")
    #p.line(temp_ulo_data.index, temp_ulo_data.values, line_width=2, color='blue', legend="Max Lows")
    #p.line(temp_dhi_data.index, temp_dhi_data.values, line_width=2, color='green', legend="Min Highs")
    p.line(temp_dlo_data.index, temp_dlo_data.values, line_width=2, color='blue', legend="Min Lows")

    p.tools.append(hover)
    return p

In [207]:
from bokeh.layouts import gridplot, column
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.plotting import figure
import pandas as pd
import sqlite3

avg_month_temp = pd.read_sql("Select Month, Year, AVG(TMAX) high, AVG(TMIN) low, AVG((TMAX+TMIN)/2) temp from weather group by 1, 2", conn)
#avg_month_temp2 = pd.read_sql("Select Month, Year, AVG((TMAX+TMIN)/2) temp from weather group by 1, 2", conn)
month_group = avg_month_temp.groupby('MONTH')

# Graph ranges
max_temp, min_temp = (weather.high.max() + 10), weather.low.min()

#Get the years for max temperatures per each calender day
weather2 = weather.loc[weather.groupby(['MONTH', 'DAY'])['Temp'].idxmax()]
hottest_years_by_day = weather2.groupby('YEAR').size()

# The function to create the AVG_MONTH_TEMP by YEAR
def temp_plots(month, data):
    # Names for Titles
    month_names = {
                  1:"January",
                  2: "February",
                  3: "March",
                  4: "April",
                  5: "May",
                  6: "June",
                  7: "July",
                  8: "August",
                  9: "September",
                  10: "October",
                  11: "November",
                  12: "December"
                  }
    #Hover tools stuff- yet to be properly implimented
#     hover = HoverTool()
#     hover.tooltips = [
#         ("date", "$index"),
#         ("(x,y)", "($x, $y)"),
#         ("radius", "@radius"),
#         ("fill color", "$color[hex, swatch]:fill_color"),
#         ("foo", "@foo"),
#         ("bar", "@bar"),
#     ]
    # rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
    # mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])
    p = figure(width=500, plot_height=250, y_range=(min_temp, max_temp), title = month_names[month])
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_color='red'
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Years'
    p.yaxis.axis_label = "Temperature"
    
    p.line(data.YEAR, data.high, line_width=2, color='red', legend="yearly high temp")
    p.line(data.YEAR, data.low, line_width=2, color='blue', legend="yearly low temp")
        
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p.legend.padding = 1

    p.tools.append(hover)

    return p
    
# Grouping 
def data_grid(lst, col=2):
    output = []
    for i in range(0, len(lst), col):
        if (i + (col - 1)) >= len(lst):
            output.append(lst[i:])
        else:
            output.append(lst[i:(i+(col))])
    return output

Loading BokehJS ...

In [218]:
years = []
for name, group in month_group:
    years.append(temp_plots(name, group))
show(column(Div(text="<h2>Quantity of Hottest Days</h2>")))
show(hottest_day_counter())
show(column(Div(text="<h2>Months Temp by Year</h2>")))
show(gridplot(data_grid(years)))
#data_grid(years, 3)

In [215]:
weather2.quantile(q=0.25)

Temp       63.5
high       71.0
low        55.0
YEAR     2007.0
MONTH       4.0
DAY         8.0
Name: 0.25, dtype: float64

In [219]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_file

# generate some synthetic time series for six different categories
cats = list("abcdef")
yy = np.random.randn(2000)
g = np.random.choice(cats, 2000)
for i, l in enumerate(cats):
    yy[g == l] += i // 2
df = pd.DataFrame(dict(score=yy, group=g))

# find the quartiles and IQR for each category
groups = df.groupby('group')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group.score > upper.loc[cat]['score']) | (group.score < lower.loc[cat]['score'])]['score']
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for keys in out.index:
        outx.append(keys[0])
        outy.append(out.loc[keys[0]].loc[keys[1]])

p = figure(tools="", background_fill_color="#efefef", x_range=cats, toolbar_location=None)

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper.score = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'score']),upper.score)]
lower.score = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'score']),lower.score)]

# stems
p.segment(cats, upper.score, cats, q3.score, line_color="black")
p.segment(cats, lower.score, cats, q1.score, line_color="black")

# boxes
p.vbar(cats, 0.7, q2.score, q3.score, fill_color="#E08E79", line_color="black")
p.vbar(cats, 0.7, q1.score, q2.score, fill_color="#3B8686", line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(cats, lower.score, 0.2, 0.01, line_color="black")
p.rect(cats, upper.score, 0.2, 0.01, line_color="black")

# outliers
if not out.empty:
    p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "white"
p.grid.grid_line_width = 2
p.xaxis.major_label_text_font_size="12pt"

show(p)

In [221]:
if __name__ == '__main__':
    main()

Loading BokehJS ...